In [1]:
from langchain.chat_models.gigachat import GigaChat
from langchain.schema import HumanMessage
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [2]:
AUTH_TOKEN = "NTlkY2MyZmItM2Q4ZC00ZWMzLWE2NjAtNTI3MzZhOTk2ZjQzOjVhZGJiZDQxLTc0YjAtNDQxNi04YjAzLTUxZDVmYTY4NTkwNw=="
PDF_PATH = "/Users/ampiro/programs/HACKATONS/DeepHackCOGers/data/raw/zsp.pdf"

In [3]:
llm = GigaChat(credentials=AUTH_TOKEN, verify_ssl_certs=False, scope='GIGACHAT_API_CORP')
embberder = GigaChatEmbeddings(
    credentials=AUTH_TOKEN, verify_ssl_certs=False, scope='GIGACHAT_API_CORP'
)

In [4]:
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 317


In [5]:
db = Chroma.from_documents(
    documents,
    embberder,
    client_settings=Settings(anonymized_telemetry=False),
)

In [6]:
qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())

In [7]:
template = "Ты полезный ассистент, который умеет переводить {input_language} на {output_language}. Переведи следующее предложение."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# get a chat completion from the formatted messages
llm.invoke(
    chat_prompt.format_prompt(
        input_language="русский",
        output_language="английский",
        text="Как представляется план",
    ).to_messages()
)

AIMessage(content='How is the plan presented?', response_metadata={'token_usage': Usage(prompt_tokens=37, completion_tokens=7, total_tokens=44), 'model_name': 'GigaChat:3.1.24.3', 'finish_reason': 'stop'}, id='run-38aca72e-c9b0-4f7d-9662-525b9a8d259c-0')

In [8]:
result = qa_chain({"query": "Show example across evaluated tasks"})["result"]
print(result)

/Users/ampiro/miniconda3/envs/cogers/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Here are the randomly chosen examples for the evaluated tasks:

Example 1: Task: Use computer
Step 1: Walk to home ofﬁce
Step 2: Walk to chair
Step 3: Find chair

Example 2: Task: Relax on sofa
Step 1: Walk to home ofﬁce
Step 2: Walk to couch
Step 3: Find couch
Step 4: Sit on couch
Step 5: Find pillow
Step 6: Lie on couch

Example 3: Task: Read book
Step 1: Walk to home ofﬁce
Step 2: Walk to couch
Step 3: Find couch
Step 4: Sit on couch
Step 5: Find pillow
Step 6: Lie on couch


In [9]:
qa_chain({"query": ""})

{'query': '', 'result': 'Не знаю'}

In [10]:
qa_chain({"query": "Что такое SARSA?"})

{'query': 'Что такое SARSA?',
 'result': 'SARSA (State-Action-Reward-State-Action) - это алгоритм обучения с подкреплением, который используется в обучении агентов для принятия решений в играх. Он основан на принципе "обучения с подкреплением", который предполагает, что агент обучается на основе своего опыта и вознаграждения, получаемого за свои действия. В SARSA агент сравнивает текущее состояние (state) с предыдущим состоянием (state), чтобы определить, какие действия привели к лучшим результатам. Затем он выбирает новое действие, основываясь на этом сравнении, и получает новое вознаграждение. Этот процесс повторяется до тех пор, пока агент не достигнет желаемого результата.'}

In [11]:
qa_chain({"query": "В чем разница между TD подходом и SARSA?"})

{'query': 'В чем разница между TD подходом и SARSA?',
 'result': 'Извините, но я не могу ответить на ваш вопрос на русском языке. Можете ли вы повторить его на английском?'}

In [12]:
qa_chain({"query": "Что сделать девушке на ужин?"})

{'query': 'Что сделать девушке на ужин?',
 'result': 'Девушке на ужин можно приготовить различные блюда, например, запеченную курицу с картофелем, овощной салат или пиццу. Также можно заказать еду на дом или пойти в ресторан.'}

In [13]:
table_request = """
Ты ассистент помогающий писать обзоры к научным статьям. Тебя попросили заполнить таблицу со столбцами, обозначающие разные важные части статьи. Сформулируй запрос на получение всей информации из статьи по названию определенного столбца.
Название интересующего столбца: среда/датасет
Не используй слово столбец в тексте запроса.
Формат ответ должен выглядеть вот так:


Запрос: "текст запроса"


"""

In [14]:
templated = """QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:""".format(
    "Как можно ",
    ""
)

KeyError: 'question'

In [15]:
qa_chain({"query": templated})

NameError: name 'templated' is not defined

In [16]:
template = """Ты делаешь обзор научных статей. Перед тобой название одной из частей обзора.
Придумай и напиши 3 УНИКАЛЬНЫХ КОРОТКИХ ВОПРОСА, которые человек может задать к данной части.
Используй непредвзятый и научнотехнический тон. Не повторяй текст.
Формат ответ должен выглядеть вот так:
"
Вопрос
...
...
Вопрос

"

Название части обзора:
{}

3 вопроса к документу на русском языке:"""

res = llm.invoke(template.format("датасет"))
print(res.content)

1. Какие данные были использованы для создания датасета?
2. Какова структура датасета и как она была разработана?
3. Какие методы и алгоритмы были применены для обработки и анализа данных в датасете?


In [17]:
command = "расскажи о чем статья абоба, обязательно скажи про dataset, llm, plan structure"

llm.invoke("""Ты многофункциональный ассистент. Ты можешь выполнять следующие действия [добавить статью в таблицу, написать короткое описание, написать короткое структурированное описание, сравнить две статьи].
Пользователь попросил тебя выполнить команду: "".
К какому из этих действий запрос пользователя ближе всего:
1. Добавить статью в таблицу
2. Написать СТРУКТУРИРОВАННЫЙ обзор на статью
3. Написать обзор на статью
4. Сравнить две статьи

Ответь только номером действия из списка. Номер ближайшего к завпросу действия:
""")

AIMessage(content='3', response_metadata={'token_usage': Usage(prompt_tokens=148, completion_tokens=3, total_tokens=151), 'model_name': 'GigaChat:3.1.24.3', 'finish_reason': 'stop'}, id='run-17eea641-0780-433a-9013-53596fb23196-0')

In [23]:
import re
s = "1. Написать обзор на статью"
print()


1


{}